### Ferret Explainability Tutorial

This notebook is based around the tutorial provided by the package authors [here](https://github.com/g8a9/ferret). 

As with the other notebooks in this repo, we applied this to a patient safety set of fictional examples, but these steps can be easily applied to any classification task.

# IMPORTANT - use 39_prompt venv

In [1]:
# to add a pipeline using ferret explainability

import sys

import numpy as np
import torch
from ferret import Benchmark, LIMEExplainer, SHAPExplainer
from IPython.display import display
from transformers import AutoModelForSequenceClassification, AutoTokenizer

sys.path.append("../")
# load custom roberta sequence classifier that uses the same averaging as declutr etc
from models.transformer_plms.model_utils.roberta_mean_classifier import MeanRobertaForSequenceClassification
from models.transformer_plms.model_utils.bert_mean_classifier import MeanBertForSequenceClassification

/mnt/sdc/niallt/venvs/39_prompt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


__WARNING__ This framework will have been tested and designed to work with the AutoModelForSequenceClassification class - so make sure to load in that style of model to avoid issue

In [14]:
cache_dir = None
model_dir = "/mnt/sdc/niallt/saved_models/pseudo_classification_tasks/mimic/ckpts/transformers/embedding_analysis/icd9-triage/fewshot_200/mimic-roberta-base/mlm_only/finetuned_plm/checkpoint-875/"

In [15]:
# load in model and tokenizer
# model = AutoModelForSequenceClassification.from_pretrained(
#     f"{model_dir}", cache_dir=cache_dir
# )
# tokenizer = AutoTokenizer.from_pretrained(f"{model_dir}")

# NOT SURE IF CUSTOM WILL WORK
model = MeanRobertaForSequenceClassification.from_pretrained(
    f"{model_dir}", cache_dir=cache_dir
)
tokenizer = AutoTokenizer.from_pretrained(f"{model_dir}")


In [5]:
model

MeanRobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [6]:
tokenizer

RobertaTokenizerFast(name_or_path='/mnt/sdc/niallt/saved_models/pseudo_classification_tasks/mimic/ckpts/transformers/embedding_analysis/icd9-triage/fewshot_200/mimic-roberta-base/mlm_only/finetuned_plm/checkpoint-875/', vocab_size=50265, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)})

## Explain a single instance

In [16]:
bench = Benchmark(model, tokenizer)

In [8]:
bench

In [17]:
# example_text = (
# "Patient was left waiting with a very high blood pressure for longer than advised. "
# "Patient seemed very agitated by the experience"
# )
example_text = "patient presented with high blood pressure and heart palpitations"

example_texts = [
    "patient presented with high blood pressure and heart palpitations",
    "severe breathing problems",
    "infant with fever and cough",
]

In [18]:
explanations = bench.explain(example_text, target=0)

In [11]:
explanations

[Explanation(text='patient presented with high blood pressure and heart palpitations', tokens=['<s>', 'patient', 'Ġpresented', 'Ġwith', 'Ġhigh', 'Ġblood', 'Ġpressure', 'Ġand', 'Ġheart', 'Ġpal', 'pit', 'ations', '</s>'], scores=array([ 8.54996276e-09, -1.87560423e-02, -5.40347248e-02, -1.58565061e-02,
         2.05880622e-02,  3.61689867e-02,  2.17179354e-01, -3.30659843e-02,
         2.88370139e-01,  6.19193469e-02,  2.48170144e-01,  5.89060840e-03,
        -9.28281671e-08]), explainer='Partition SHAP', target=0),
 Explanation(text='patient presented with high blood pressure and heart palpitations', tokens=['<s>', 'patient', 'Ġpresented', 'Ġwith', 'Ġhigh', 'Ġblood', 'Ġpressure', 'Ġand', 'Ġheart', 'Ġpal', 'pit', 'ations', '</s>'], scores=array([ 0.        , -0.03768368,  0.0183629 , -0.07176974, -0.02019631,
        -0.04068256,  0.15068074, -0.04407334,  0.23409157,  0.08034562,
         0.21731716,  0.08479639,  0.        ]), explainer='LIME', target=0),
 Explanation(text='patient pre

In [19]:
bench.show_table(explanations)

Token,patient,Ġpresented,Ġwith,Ġhigh,Ġblood,Ġpressure,Ġand,Ġheart,Ġpal,pit,ations
Partition SHAP,-0.02,-0.05,-0.02,0.02,0.04,0.22,-0.03,0.29,0.06,0.25,0.01
LIME,0.00,-0.02,-0.03,-0.09,-0.07,0.18,0.02,0.23,0.13,0.12,0.11
Gradient,0.09,0.07,0.05,0.05,0.10,0.14,0.06,0.09,0.08,0.13,0.05
Gradient (x Input),0.06,-0.04,-0.02,-0.01,0.17,-0.17,-0.01,0.23,0.03,-0.16,-0.04
Integrated Gradient,0.04,0.10,-0.02,0.19,0.18,-0.14,0.02,0.07,0.05,0.10,0.06
Integrated Gradient (x Input),-0.03,0.02,-0.01,-0.06,0.06,-0.05,-0.03,0.38,0.10,0.19,0.07


## Evaluate explanation of a single instance

Evaluate explanations with all the supported evaluators is straightforward. Remember to specify the `target` parameter to match the one used during the explanation!

Area Over the Perturbation Curve (AOPC) Comprehensiveness (`aopc_compr`), AOPC Sufficiency (`aopc_suff`) and Correlation with Leave-One-Out scores (`taucorr_loo`) are three measures of faithfulness.

- **AOPC Comprehensiveness**. Comprehensiveness measures the drop in the model probability if the relevant tokens of the explanations are removed. We measure comprehensiveness via the Area Over the Perturbation Curve by progressively considering the most $k$ important tokens, with $k$ from 1 to #tokens (as default) and then averaging the result. The higher the value, the more the explainer is able to select the relevant tokens for the prediction.

- **AOPC Sufficiency**. Sufficiency captures if the tokens in the explanation are sufficient for the model to make the prediction. As for comprehensiveness, we use the AOPC score.

- **Correlation with Leave-One-Out scores**. We first compute the leave-one-out scores by computing the prediction difference when one feature at the time is omitted. We then measure the Spearman correlation with the explanations.

In [20]:
explanation_evaluations = bench.evaluate_explanations(explanations, target=0)

In [21]:
bench.show_evaluation_table(explanation_evaluations)

,aopc_compr,aopc_suff,taucorr_loo
Partition SHAP,0.72,-0.02,0.16
LIME,0.59,-0.03,0.45
Gradient,0.43,0.09,-0.16
Gradient (x Input),-0.03,-0.00,0.02
Integrated Gradient,0.27,0.36,0.20
Integrated Gradient (x Input),0.49,-0.02,0.42


Given ground truth explanations - which are ultimately the tokens from the passage that are deemed as important to the decision in this case, we can calcualte some futher metrics. The explanations are provided as a vector similar to attention mask, with 0s for non-important tokens and 1s for important tokens. 

The bench class will (I think) expect a human rationale of the `length(tokenizer.encode(text)-2)`, essentially the number of tokens minus the bos and eos tokens.

In [11]:
tokenizer.encode(example_text)

[0, 133, 3186, 1064, 66, 9, 3267, 8, 2263, 49, 2985, 2]

In [12]:
len(tokenizer.encode(example_text))

12

### Function to look at multiple models

In [26]:
def plot_multiple_explained_models(model_name_or_path, sample_text, target_class=0):
    
    
    
    
    print(f"##### Working on model {model_dir} #####")
    # load in model and tokenizer
    model = MeanRobertaForSequenceClassification.from_pretrained(
    f"{model_name_or_path}", cache_dir=None
    )
    tokenizer = AutoTokenizer.from_pretrained(f"{model_dir}")
    
    # create benchmark
    bench = Benchmark(model, tokenizer)
    # pass example
    explanations = bench.explain(example_text, target=target_class)
    return bench.show_table(explanations)
    


In [27]:
model_paths = ["/mnt/sdc/niallt/saved_models/pseudo_classification_tasks/mimic/ckpts/transformers/embedding_analysis/icd9-triage/fewshot_200/roberta-base/declutr/2_anch_2_pos_min_1024/finetuned_plm/checkpoint-875",
               "/mnt/sdc/niallt/saved_models/pseudo_classification_tasks/mimic/ckpts/transformers/embedding_analysis/icd9-triage/fewshot_200/roberta-base/finetuned_plm/checkpoint-875",
]


example_text = "patient presented with high blood pressure and heart palpitations. They have a history of heart disease and diabetes, with family history of heart disease."    
plot_multiple_explained_models(model_paths[0], example_text, target_class=0)



##### Working on model /mnt/sdc/niallt/saved_models/pseudo_classification_tasks/mimic/ckpts/transformers/embedding_analysis/icd9-triage/fewshot_200/roberta-base/declutr/2_anch_2_pos_min_1024/finetuned_plm/checkpoint-875 #####


,patient,Ġpresented,Ġwith,Ġhigh,Ġblood,Ġpressure,Ġand,Ġheart,Ġpal,pit,ations,.,ĠThey,Ġhave,Ġa,Ġhistory,Ġof,Ġheart.1,Ġdisease,Ġand.1,Ġdiabetes,",",Ġwith.1,Ġfamily,Ġhistory.1,Ġof.1,Ġheart.2,Ġdisease.1,..1
Partition SHAP,-0.04,0.03,0.02,-0.05,-0.12,0.10,0.01,0.08,0.02,0.02,0.01,-0.00,0.00,0.00,-0.00,-0.00,0.03,0.09,-0.01,0.00,-0.12,-0.01,-0.01,-0.01,0.00,0.00,0.17,-0.04,-0.01
LIME,-0.03,0.01,0.01,-0.02,-0.16,0.14,0.05,0.08,0.01,0.02,0.00,-0.02,-0.02,0.00,0.00,-0.02,0.01,0.08,0.01,-0.02,-0.12,-0.00,-0.01,0.04,-0.00,0.01,0.10,0.01,0.00
Gradient,0.04,0.02,0.02,0.05,0.10,0.11,0.02,0.04,0.03,0.05,0.02,0.01,0.03,0.01,0.01,0.01,0.01,0.05,0.03,0.02,0.07,0.01,0.02,0.03,0.02,0.02,0.06,0.04,0.01
Gradient (x Input),-0.03,-0.05,0.01,-0.07,0.14,-0.10,0.02,0.03,0.04,0.00,0.03,-0.03,-0.01,0.01,-0.01,-0.01,-0.01,-0.03,0.00,0.02,-0.11,0.01,0.04,0.04,-0.03,0.01,0.02,-0.03,0.03
Integrated Gradient,-0.03,-0.00,0.01,0.02,-0.08,-0.03,0.01,-0.19,0.04,-0.06,0.03,-0.04,0.00,-0.01,-0.01,-0.00,-0.04,-0.13,0.03,-0.01,-0.01,0.02,0.01,-0.04,0.02,-0.01,-0.05,-0.01,0.03
Integrated Gradient (x Input),-0.02,-0.03,0.01,-0.03,0.03,0.04,0.04,0.12,0.06,-0.05,0.04,0.00,0.01,-0.00,0.02,-0.02,0.02,0.12,-0.03,0.00,-0.04,-0.00,-0.00,-0.11,-0.02,0.02,0.07,-0.03,-0.00


In [28]:
plot_multiple_explained_models(model_paths[1], example_text, target_class=0)

##### Working on model /mnt/sdc/niallt/saved_models/pseudo_classification_tasks/mimic/ckpts/transformers/embedding_analysis/icd9-triage/fewshot_200/roberta-base/declutr/2_anch_2_pos_min_1024/finetuned_plm/checkpoint-875 #####


,patient,Ġpresented,Ġwith,Ġhigh,Ġblood,Ġpressure,Ġand,Ġheart,Ġpal,pit,ations,.,ĠThey,Ġhave,Ġa,Ġhistory,Ġof,Ġheart.1,Ġdisease,Ġand.1,Ġdiabetes,",",Ġwith.1,Ġfamily,Ġhistory.1,Ġof.1,Ġheart.2,Ġdisease.1,..1
Partition SHAP,0.06,0.02,0.02,0.01,-0.02,0.13,0.00,0.11,0.03,0.04,0.01,0.02,0.00,-0.00,-0.00,-0.00,-0.00,0.12,-0.01,0.01,-0.22,-0.00,0.00,0.01,-0.00,0.01,0.12,-0.01,0.00
LIME,-0.00,0.02,0.01,-0.00,-0.04,0.05,-0.01,0.16,0.01,0.04,-0.01,-0.00,-0.01,-0.00,0.00,0.02,0.00,0.12,-0.01,0.02,-0.16,0.04,-0.01,0.01,0.01,0.05,0.14,-0.02,-0.00
Gradient,0.02,0.03,0.02,0.02,0.05,0.08,0.03,0.09,0.03,0.06,0.03,0.02,0.02,0.01,0.01,0.02,0.01,0.06,0.04,0.02,0.10,0.02,0.02,0.03,0.01,0.01,0.05,0.03,0.01
Gradient (x Input),-0.01,-0.00,-0.01,-0.03,0.10,-0.02,0.06,-0.01,-0.02,-0.13,-0.01,0.01,-0.06,0.01,0.03,-0.04,0.01,0.03,-0.02,-0.00,0.11,0.01,0.03,-0.05,-0.03,0.03,0.02,-0.00,-0.01
Integrated Gradient,-0.06,-0.10,-0.00,-0.04,0.02,-0.05,-0.01,-0.11,-0.01,0.03,-0.02,0.07,-0.07,0.01,-0.00,-0.00,0.00,-0.02,0.01,-0.04,0.02,-0.01,0.02,-0.04,-0.02,0.01,-0.03,-0.04,-0.09
Integrated Gradient (x Input),0.00,0.07,0.02,0.03,0.02,0.04,0.02,0.07,0.03,0.01,0.02,0.06,0.05,0.01,0.02,-0.02,0.04,0.07,-0.02,0.00,-0.04,0.00,0.01,-0.01,-0.01,0.03,0.05,-0.02,0.20


# look at real mimic example

In [24]:
# data_dir = "/mnt/sdc/niallt/mimic_iii/processed/HADM_ID_split/"
# data_dir = "/mnt/sdc/niallt/mimic_iii/processed/HADM_ID_split/pseudo_classification/class_reduced_8/fewshot_200/"
from datasets import load_dataset
data_dir = "/mnt/sdc/niallt/mimic_iii/processed/HADM_ID_split/icd9-triage/no_category_in_text/fewshot_16"
cache_dir = "/mnt/sdc/niallt/.cache/"

# load as a dataset
dataset = load_dataset("csv", 
                        data_files = {"train":f"{data_dir}/train.csv",
                                        "valid":f"{data_dir}/test.csv"},
                        cache_dir = cache_dir)

100%|██████████| 2/2 [00:00<00:00, 182.64it/s]


In [27]:
idx = 10
example_text = dataset["train"]["text"][idx]
# target 
example_label = dataset["train"]["label"][idx]

In [34]:
# truncate the example text to 50 tokens - its absolutely un-interpretable for long sequences
example_text = " ".join(example_text.split()[:50])

In [33]:
example_text

': : : sex: m history: baby boy is a and / weeks gestational age infant admitted with antenatal diagnosis of intracranial hemorrhagic event. maternal history: mom is a -year-old gravida , para , now mom with a past medial history notable for asthma, for cigarette smoking pack per day, marijuana use and alcohol use, all prior to months gestation when she became aware of pregnancy. prenatal screens: blood type b positive, antibody negative, rpr nonreactive, group beta strep negative, hepatitis b surface antigen negative, and rubella immune. antenatal history: mom was a late registrant to prenatal care, with pregnancy not diagnosed until months. estimated date of delivery based on the to month ultrasound on . the initial ultrasound showed ventriculomegaly and a porencephalic cyst. subsequent prenatal mri showed the following: . left ventriculomegaly extended across baseline with associated porencephalic cyst. . absent corpus callosum. . compression of posterior fossa and effacement of ext

In [28]:
example_label

1

In [35]:
explanations = bench.explain(example_text, target=example_label)

In [31]:
explanations

[Explanation(text=": : : sex: m history: baby boy is a and / weeks gestational age infant admitted with antenatal diagnosis of intracranial hemorrhagic event. maternal history: mom is a -year-old gravida , para , now mom with a past medial history notable for asthma, for cigarette smoking pack per day, marijuana use and alcohol use, all prior to months gestation when she became aware of pregnancy. prenatal screens: blood type b positive, antibody negative, rpr nonreactive, group beta strep negative, hepatitis b surface antigen negative, and rubella immune. antenatal history: mom was a late registrant to prenatal care, with pregnancy not diagnosed until months. estimated date of delivery based on the to month ultrasound on . the initial ultrasound showed ventriculomegaly and a porencephalic cyst. subsequent prenatal mri showed the following: . left ventriculomegaly extended across baseline with associated porencephalic cyst. . absent corpus callosum. . compression of posterior fossa and

In [36]:
bench.show_table(explanations)

### Plausability

In [23]:
explanation_evaluations = bench.evaluate_explanations(
    explanations, target=1, human_rationale=[0, 1, 0, 0], top_k_rationale=1
)
bench.show_evaluation_table(explanation_evaluations)

Explanation eval:   0%|          | 0/6 [00:00<?, ?it/s]

ValueError: 

Plausibility evaluates how well the explanation agree with human rationale. We evaluate plausibility via 
Area Under the Precision Recall curve (AUPRC) (`auprc_plau`),  token-level f1-score (`token_f1_plau`) and average Intersection-Over-Union (`IOU`) at the token level (`token_iou_plau`).

- **Area Under the Precision Recall curve (AUPRC)** is computed by sweeping a threshold over token scores.
- **Token-level f1-score** and the **`**average Intersection-Over-Union** consider discrete rationales.
We derive a discrete rationale by taking the top-k values. K in the example is set to 1.
- **Token-level f1-score** is the token-level F1 scores derived from the token-level precision and recall. 
- **Intersection-Over-Union (IOU)** is the size of the overlap of the tokens they cover divided by the size of their union.

When the set of human rationales for the dataset is available, K is set as the average rationale length (as in ERASER)

**Interface to individual explainers**

You can also use individual explainers using an object oriented interface.

### LIME

In [ ]:
lime_exp = LIMEExplainer(model, tokenizer)
lime_values = lime_exp(example_text, target=1)

In [ ]:
lime_values

### SHAP

In [ ]:
shap_exp = SHAPExplainer(model, tokenizer)
shap_values = shap_exp(example_text)

In [ ]:
shap_values

multiple examples

In [13]:
import numpy as np
bench = Benchmark(model, tokenizer)

# Compute and average evaluation scores one of the supported dataset
samples = np.arange(20)
hatexdata = bench.load_dataset("hatexplain")


Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 992.50it/s]


Dataset hatexplain downloaded and prepared to /home/niallt/.cache/huggingface/datasets/hatexplain/plain_text/1.0.0/df474d8d8667d89ef30649bf66e9c856ad8305bef4bc147e8e31cbdf1b8e0249. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 386.77it/s]


In [22]:
# get attriutes of hatexdata
hatexdata.__dict__["train_dataset"][0]

{'id': '23107796_gab',
 'annotators': {'label': [0, 2, 2],
  'annotator_id': [203, 204, 233],
  'target': [['Hindu', 'Islam'],
   ['Hindu', 'Islam'],
   ['Hindu', 'Islam', 'Other']]},
 'rationales': [[0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   1,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   1,
   1,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0]],
 'post_tokens': ['u',
  'really',
  'think',
  'i',
  'would',
  'not',
  'have',
  'been',
  'raped',
  'by',
  'feral',
  'hindu',
  'or',
  'muslim',
  'back',
  'in',
  'india',
  'or',
  'bangladesh',
  'and',
  'a',
  'neo',
  'nazi',
  'would',
  'rape',
  'me',
  'as',
  'well',
  'just',
  'to',
  'see',
  'me',
  'cry']}

In [ ]:
sample_evaluations =  bench.evaluate_samples(hatexdata, samples)

# Pretty-print the results
bench.show_samples_evaluation_table(sample_evaluations)

In [20]:
# have to make own dataset class

from typing import List

import numpy as np
import pytreebank

from ferret.datasets import BaseDataset

TRAIN_SET = "train"
VALIDATION_SET = "validation"
TEST_SET = "test"

NONE_RATIONALE = []

In [ ]:
class MimicTriage(BaseDataset):

    NAME = "MimicTriage"
    avg_rationale_size = 7
    # np.mean([sum(self._get_rationale(i, split_type="train")[self._get_ground_truth(i, split_type="train")]) for i in range(self.len("train"))])

    def __init__(self, tokenizer):
        from datasets import load_dataset

        data_dir = "/mnt/sdc/niallt/mimic_iii/processed/HADM_ID_split/icd9-triage/no_category_in_text/fewshot_16/"


        # load as a dataset
        dataset = load_dataset("csv", 
                                data_files = {"train":f"{data_dir}/train.csv",
                                                "valid":f"{data_dir}/valid.csv",
                                                "test":f"{data_dir}/test.csv"},
                                cache_dir = None)

        self.train_dataset = dataset["train"]
        self.validation_dataset = dataset["validation"]
        self.test_dataset = dataset["test"]
        self.tokenizer = tokenizer
        self.top_k_hard_rationale = 7
        self.classes = [0, 1, 2, 3,4,5,6]

    def __len__(self):
        # We use the TEST_SET as default
        return self.len()

    def len(self, split_type: str = TEST_SET):
        if split_type == TRAIN_SET:
            return len(self.train_dataset)
        elif split_type == VALIDATION_SET:
            return len(self.validation_dataset)
        elif split_type == TEST_SET:
            return len(self.test_dataset)
        else:
            raise ValueError(
                f"{split_type} not supported as split_type. Specify one among: train, validation or test."
            )

    def _get_item(self, idx: int, split_type: str = TEST_SET):
        if isinstance(idx, int):
            if split_type == TRAIN_SET:
                item_idx = self.train_dataset[idx]
            elif split_type == VALIDATION_SET:
                item_idx = self.validation_dataset[idx]
            elif split_type == TEST_SET:
                item_idx = self.test_dataset[idx]
            else:
                raise ValueError(
                    f"{split_type} not supported as split_type. Specify one among:  train, validation or test."
                )
            return item_idx
        elif isinstance(idx, dict):
            return idx
        else:
            raise ValueError()

    def __getitem__(self, idx):
        # We use the TEST_SET as default
        return self.get_instance(idx)

    def get_instance(self, idx, split_type: str = TEST_SET, rationale_union=True):
        item_idx = self._get_item(idx, split_type)
        text = self._get_text(item_idx)
        tokens = (
            [self.tokenizer.cls_token]
            + self.tokenizer.tokenize(text)
            + [self.tokenizer.sep_token]
        )
        rationale = self._get_rationale(item_idx, split_type, rationale_union)
        true_label = self._get_ground_truth(item_idx, split_type)
        return {
            "text": text,
            "tokens": tokens,
            "rationale": rationale,
            "label": true_label,
        }

    def _get_text(self, idx, split_type: str = TEST_SET):
        item_idx = self._get_item(idx, split_type)
        post_tokens = item_idx["post_tokens"]
        text = " ".join(post_tokens)
        return text

    def _get_rationale(self, idx, split_type: str = TEST_SET, rationale_union=True):
        item_idx = self._get_item(idx, split_type)
        word_based_tokens = item_idx["post_tokens"]

        # All hatexplain rationales are defined for the label, only for hatespeech or offensive classes
        rationale_label = self._get_ground_truth(idx, split_type)

        rationale_by_label = [NONE_RATIONALE for c in self.classes]
        if "rationales" in item_idx:
            rationales = item_idx["rationales"]
            if len(rationales) > 0 and isinstance(rationales[0], list):
                # It is a list of lists
                if rationale_union:
                    # We get the union of the rationales.
                    rationale = [any(each) for each in zip(*rationales)]
                    rationale = [int(each) for each in rationale]
                else:
                    # We return all of them (deprecated)
                    rationale_by_label[rationale_label] = [
                        self.get_true_rationale_from_words_to_tokens(
                            word_based_tokens, rationale
                        )
                        for rationale in rationales
                    ]
                    return rationale_by_label
            else:
                rationale = rationales
        rationale_by_label[
            rationale_label
        ] = self.get_true_rationale_from_words_to_tokens(word_based_tokens, rationale)

        return rationale_by_label

    def _get_ground_truth(self, idx, split_type: str = TEST_SET):
        item_idx = self._get_item(idx, split_type)
        labels = item_idx["annotators"]["label"]
        # Label by majority voting
        return max(set(labels), key=labels.count)

    def get_true_rationale_from_words_to_tokens(
        self, word_based_tokens: List[str], words_based_rationales: List[int]
    ):
        return super().get_true_rationale_from_words_to_tokens(
            word_based_tokens, words_based_rationales
        )
